1. My contribution mainly lies in extracting holiday features. 
2. I used Fitter to view the data distribution of the target column and used statistical indicators to check the deviation between predicted values and true values.
3. It is a stable model. When I tried changing the random number and cross validation fold, the difference in results was very small.
4. I haven't programmed for many years. I have been learning Python for the past few months, but I am still not very proficient. If it is helpful, please upvote. Thanks to everyone.

In [ ]:
!pip install -U --q ./input/fitter-1-71/fitter-1.7.1-py3-none-any.whl
!pip install -U --q pycountry

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error 
import holidays
import pycountry

from fitter import Fitter
from scipy import stats
import shap
from datetime import datetime, timedelta

SEED = 42
n_splits = 5
n_estimators=5000
early_stopping_rounds = 100
FE_holidays = True

In [ ]:
train_data = pd.read_csv(r"./input/playground-series-s5e1/train.csv")
test_data = pd.read_csv(r"./input/playground-series-s5e1/test.csv")
data = pd.read_csv(r"./input/playground-series-s5e1/sample_submission.csv")

print("train_data shape :",train_data.shape)
print("test_data shape :",test_data.shape)
print("data shape :",data.shape)

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

In [ ]:
train_data.isna().sum().sort_values(ascending=False)

In [ ]:
train_data['country'].value_counts()

In [ ]:
train_data = train_data.drop_duplicates()
train_data = train_data.dropna()
print("train_data shape :",train_data.shape)

In [ ]:
test_data.head()

In [ ]:
test_data.isna().sum().sort_values(ascending=False)

In [ ]:
train_data['last'] = train_data.groupby(['country', 'store', 'product'])['num_sold'].transform('last')
#train_data['mean'] = train_data.groupby(['country', 'store', 'product'])['num_sold'].transform('mean')

In [ ]:
mean_mapping = train_data.drop_duplicates(['country', 'store', 'product'])[['country', 'store', 'product', 'last']]
test_data = test_data.merge(mean_mapping, on=['country', 'store', 'product'], how='left')

#mean_mapping = train_data.drop_duplicates(['country', 'store', 'product'])[['country', 'store', 'product', 'mean']]
#test_data = test_data.merge(mean_mapping, on=['country', 'store', 'product'], how='left')

train_data.isna().sum().sort_values(ascending=False)

In [ ]:
#print(train_data['last'].value_counts())

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

train_data['Year'] = train_data['date'].dt.year
train_data['Quarter'] = train_data['date'].dt.quarter
train_data['Month'] = train_data['date'].dt.month
train_data['Day'] = train_data['date'].dt.day
train_data['day_of_week'] = train_data['date'].dt.day_name()
train_data['week_of_year'] = train_data['date'].dt.isocalendar().week

#train_data['day_sin'] = np.sin(2 * np.pi * train_data['Day'] / 365.0)
train_data['day_cos'] = np.cos(2 * np.pi * train_data['Day'] / 365.0)
train_data['month_sin'] = np.sin(2 * np.pi * train_data['Month'] / 12.0)
train_data['month_cos'] = np.cos(2 * np.pi * train_data['Month'] / 12.0)
train_data['year_sin'] = np.sin(2 * np.pi * train_data['Year'] / 7.0)
train_data['year_cos'] = np.cos(2 * np.pi * train_data['Year'] / 7.0)
train_data['Group']=(train_data['Year']-2010)*48+train_data['Month']*4+train_data['Day']//7

test_data['Year'] = test_data['date'].dt.year
test_data['Quarter'] = test_data['date'].dt.quarter
test_data['Month'] = test_data['date'].dt.month
test_data['Day'] = test_data['date'].dt.day
test_data['day_of_week'] = test_data['date'].dt.day_name()
test_data['week_of_year'] = test_data['date'].dt.isocalendar().week

#test_data['day_sin'] = np.sin(2 * np.pi * test_data['Day'] / 365.0)
test_data['day_cos'] = np.cos(2 * np.pi * test_data['Day'] / 365.0)
test_data['month_sin'] = np.sin(2 * np.pi * test_data['Month'] / 12.0)
test_data['month_cos'] = np.cos(2 * np.pi * test_data['Month'] / 12.0)
test_data['year_sin'] = np.sin(2 * np.pi * test_data['Year'] / 7.0)
test_data['year_cos'] = np.cos(2 * np.pi * test_data['Year'] / 7.0)
test_data['Group']=(test_data['Year']-2010)*48+test_data['Month']*4+test_data['Day']//7



In [ ]:
train_data['Quarter'] = train_data['Quarter'].astype('str')
train_data['Month'] = train_data['Month'].astype('str')
train_data['day_of_week'] = train_data['day_of_week'].astype('str')
train_data['week_of_year'] = train_data['week_of_year'].astype('str')

test_data['Quarter'] = test_data['Quarter'].astype('str')
test_data['Month'] = test_data['Month'].astype('str')
test_data['day_of_week'] = test_data['day_of_week'].astype('str')
test_data['week_of_year'] = test_data['week_of_year'].astype('str')

In [ ]:
#train_data['country_store'] = train_data['country'] + "_" + train_data['store'] 
#train_data['country_product'] = train_data['country'] + "_" + train_data['product']
#train_data['store_product'] = train_data['store'] + "_" + train_data['store'] 
#train_data['country_store_product'] = train_data['country'] + "_" + train_data['store'] + "_" + train_data['product']

In [ ]:
#test_data['country_store'] = test_data['country'] + "_" + test_data['store'] 
#test_data['country_product'] = test_data['country'] + "_" + test_data['product']
#test_data['store_product'] = test_data['store'] + "_" + test_data['store'] 
#test_data['country_store_product'] = test_data['country'] + "_" + test_data['store'] + "_" + test_data['product']

In [ ]:
def get_holiday_name(country_code, date_obj):
    country_holiday = holidays.CountryHoliday(country_code, years=date_obj.year)
    return country_holiday.get(date_obj)

def get_country_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_2  
    except KeyError:
        print(f"Unknown Country: {country_name}")
        return None

def get_holiday_for_row(row):
    country_code = get_country_code(row['country'])
    if country_code is None:
        return 'Unknown Country'
    
    try:
        date_obj = row['date']
    except ValueError:
        print(f"Invalid Date: {row['date']}")
        return 'Invalid Date'
    
    return get_holiday_name(country_code, date_obj)



In [ ]:
def get_holiday_pre1_for_row(row):
    country_code = get_country_code(row['country'])
    if country_code is None:
        return 'Unknown Country'
    
    try:
        date_obj = row['date']+ timedelta(days=1)
    except ValueError:
        print(f"Invalid Date: {row['date']}")
        return 'Invalid Date'
    
    return get_holiday_name(country_code, date_obj)

In [ ]:
def get_holiday_pre2_for_row(row):
    country_code = get_country_code(row['country'])
    if country_code is None:
        return 'Unknown Country'
    
    try:
        date_obj = row['date']+ timedelta(days=2)
    except ValueError:
        print(f"Invalid Date: {row['date']}")
        return 'Invalid Date'
    
    return get_holiday_name(country_code, date_obj)

In [ ]:
def get_holiday_pre3_for_row(row):
    country_code = get_country_code(row['country'])
    if country_code is None:
        return 'Unknown Country'
    
    try:
        date_obj = row['date']+ timedelta(days=3)
    except ValueError:
        print(f"Invalid Date: {row['date']}")
        return 'Invalid Date'
    
    return get_holiday_name(country_code, date_obj)

In [ ]:
if FE_holidays:
    train_data['holidays_name'] = train_data.apply(get_holiday_for_row, axis=1)
    test_data['holidays_name'] = test_data.apply(get_holiday_for_row, axis=1)
    print(train_data['holidays_name'].value_counts() )
    
    train_data['holidays_name_pre1'] = train_data.apply(get_holiday_pre1_for_row, axis=1)
    test_data['holidays_name_pre1'] = test_data.apply(get_holiday_pre1_for_row, axis=1)

    #train_data['holidays_name_pre2'] = train_data.apply(get_holiday_pre2_for_row, axis=1)
    #test_data['holidays_name_pre2'] = test_data.apply(get_holiday_pre2_for_row, axis=1)

    #train_data['holidays_name_pre3'] = train_data.apply(get_holiday_pre3_for_row, axis=1)
    #test_data['holidays_name_pre3'] = test_data.apply(get_holiday_pre3_for_row, axis=1)

In [ ]:
train_data.drop('date',axis=1,inplace=True)
test_data.drop('date',axis=1,inplace=True)

In [ ]:
print(train_data.info())

train_data = train_data.drop('id', axis = 1)
num_cols = list(train_data.select_dtypes(exclude=['object']).columns.difference(['num_sold']))
cat_cols = list(train_data.select_dtypes(include=['object']).columns)

num_cols_test = list(test_data.select_dtypes(exclude=['object']).columns.difference(['id']))
cat_cols_test = list(test_data.select_dtypes(include=['object']).columns)

print('num_cols:',num_cols)
print('cat_cols:',cat_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
label_encoders = {col: LabelEncoder() for col in cat_cols}

# Apply LabelEncoder to each categorical column
for col in cat_cols:
    combined_data = pd.concat([train_data[col], test_data[col]])
    le = LabelEncoder()
    le.fit(combined_data)
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])
    

In [ ]:
from sklearn.model_selection import train_test_split
#train_data['num_sold'] = np.log1p(train_data['num_sold'])
X = train_data.drop(['num_sold'], axis=1)
y = train_data['num_sold']-train_data['last']
test = test_data.drop(['id'],axis=1)


f = Fitter(y, distributions=['norm', 'lognorm', 'laplace', 'expon', 'gamma'])
f.fit()
print('Distribution of original training data:')
f.summary()
lognorm_params = f.fitted_param['lognorm']

In [ ]:

# Split datainto training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define MAPE metric
def mape(y_true, y_pred):
    return mean_squared_error (y_true, y_pred)

In [ ]:
kf = KFold(n_splits, shuffle=True, random_state=SEED)
kf_splits = kf.split(X)
scores1 = []
train_preds1 = []
test_preds1 = []
models = []

lgbm_params1 = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': n_estimators,
    'learning_rate': 0.08,
    'max_depth': 13,
    'reg_alpha': 0.01,
    'lambda_l2': 0.01,  
    'min_child_samples' : 32,
    'colsample_bytree': 0.93,
    'subsample': 0.7, 
    'seed': SEED,
    'verbose': -1,
    'device' : 'cpu' 
}

for i, (train_idx, val_idx) in enumerate(kf_splits):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    #train_lgbm_dataset = lgb.Dataset(X_train_fold, label=y_train_fold, categorical_feature=cat_cols)
    #valid_lgbm_dataset = lgb.Dataset(X_val_fold, label=y_val_fold, categorical_feature=cat_cols)
    
    callbacks = [lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False)]
    model = lgb.LGBMRegressor(**lgbm_params1)
    model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], categorical_feature=cat_cols,eval_metric='l2', callbacks=callbacks)
    models.append(model)
    
    val_pred = np.maximum(model.predict(X_val_fold, num_iteration=model.best_iteration_),0)
    score = mape(y_val_fold, val_pred)
    scores1.append(score)
    
    test_pred = np.maximum(model.predict(test, num_iteration=model.best_iteration_),0)
    test_preds1.append(test_pred)
    
    print(f'LightGBM Fold {i + 1} mean_squared_error : {score:.5f}')
print(f'LightGBM mean_squared_error : {np.mean(scores1):.5f};');


In [ ]:
print('best_iteration: ',model.best_iteration_)

In [ ]:
for i, model in enumerate(models):
    lgb.plot_importance(model, max_num_features=50, importance_type='split') 
    plt.title(f"Feature Importances for Model {i+1}")
    plt.show()

In [ ]:
#y_preds = (np.mean(test_preds1, axis=0)+np.mean(test_preds2, axis=0))/2
y_preds = np.mean(test_preds1, axis=0)
y_preds = y_preds+test["last"].fillna(5)
print(y_preds)
print('predict mean before:',y_preds.mean())
print('predict median before:',np.median(y_preds))
y_preds = np.round(y_preds*1.004)
print('predict mean after:',y_preds.mean())
print('predict median after:',np.median(y_preds))

y_preds = np.clip(y_preds, 5, 5939)


In [ ]:
f = Fitter(y_preds, distributions=['norm', 'lognorm', 'laplace', 'expon', 'gamma'])
f.fit()
print('Distribution of predicting data:')
f.summary()   
norm_params = f.fitted_param['norm']

In [ ]:
# Save predictions for submission
submission = pd.DataFrame({'id': test_data['id'], 'num_sold': y_preds})
submission.to_csv('submission.csv', index=False)
print(submission.head())